# Revisando os Conceitos da Aula Anterior


## ETL e ELT


#### ETL -> Extract Transform and Load

- Dados ingeridos já de acordo com sua finalidade
- Sempre que adquiro um dado já aplico as transformações de negócio necessárias
- Necessita de um conhecimento Prévio da Finalidade do Dado

#### ELT -> Extract Load and Transform

- Todo dado é válido
- Armazenamento do dado em seu estado original
- Não necessitamos conhecer a finalidade para a aquisição de um dado
- Torna mais flexível e reaproveitável o uso do dado ingerido
- Permite níveis de refinamento diferentes do dado disponíveis no ambiente


## APIs

- Biblioteca Python Requests

  - > %pip install requests
  - > import requests

- APIs são interfaces que nos permitem conversar com outros sistemas e contextos

- Principais verbos HTTP

  - GET
  - POST
  - PUT
  - PATCH
  - DELETE

- Introdução aos Códigos de Resposta HTTP
  - 200 - 299 - OK
  - 400 - 499 - Erro por parte do Cliente
  - 500 - 599 - Erro por parte do Servidor


# EXTRAÇÃO DE DADOS I - AULA 02


In [ ]:
%pip install requests

In [ ]:
import requests

In [ ]:
response = requests.get(url="https://fakestoreapi.com/products")

### Formatando Resposta de uma Requisição

Neste exemplo nosso objetivo é obter a resposta de nossa requisição como um dict


In [ ]:
import json

text_content = response.text

print(f"O tipo da resposta é {type(text_content)}")

json_content_parsed = json.loads(text_content)

print(f"O tipo após a conversão utilizando json.loads é {type(json_content_parsed)}")

In [ ]:
json_content = response.json()
print(f"O tipo de dado utilizando a função json é {type(json_content)}")

### Tratando Retornos com Erro

- Códigos 500 e 400


In [ ]:
response = requests.get(url="https://fakestoreapi.com/productsaaaaaaaa")
print(f"O código de retorno é: {response.status_code}")

In [ ]:
## Quando tentamos utilizar o método json porém o retorno é diferente de um json, temos uma excessão, 
## porém ainda é possível utilizar o método text para obter o resultado da execução
response.json()

In [ ]:
response.text

In [ ]:
def gerencia_requisicao(url: str, endpoint: str, current_retry: int=0, max_retries: int=0):

    urls = [
        "https://fakestoreapi.com/productsa", 
        "https://fakestoreapi.com/productsb", 
        "https://fakestoreapi.com/products"
    ]
    
    response = requests.get(url=urls[current_retry])
    
    if (response.status_code >= 400 and response.status_code <= 499) or (response.status_code >= 500 and response.status_code <= 599):
        if current_retry < max_retries:
            return gerencia_requisicao(
                url=url,
                endpoint=endpoint,
                current_retry=current_retry+1,
                max_retries=max_retries
            )
        else:
            return "error", {
                "code": response.status_code,
                "message":  response.text
            }


    return "success", response.text

In [ ]:
import json

response = gerencia_requisicao(
    url="https://fakestoreapi.com",
    endpoint="products",
    max_retries=3
)

In [ ]:
import time

def gerencia_requisicao_com_espera(url: str, endpoint: str, current_retry: int=0, max_retries: int=0):

    urls = [
        "https://fakestoreapi.com/productsa", 
        "https://fakestoreapi.com/productsb", 
        "https://fakestoreapi.com/products"
    ]
    
    response = requests.get(url=urls[current_retry])
    
    if (response.status_code >= 400 and response.status_code <= 499) or (response.status_code >= 500 and response.status_code <= 599):
        if current_retry < max_retries:

            time.sleep(5)
            
            return gerencia_requisicao(
                url=url,
                endpoint=endpoint,
                current_retry=current_retry+1,
                max_retries=max_retries
            )
        else:
            return "error", {
                "code": response.status_code,
                "message":  response.text
            }


    return "success", response.text

In [ ]:
import datetime

gerencia_requisicao_com_espera(
    url="https://fakestoreapi.com",
    endpoint="products",
    max_retries=3
)

### JSON

Listas = []
Objetos = {}
Valores = 1, "", null

{
"id": 1,
"name": "John",
"notas": [5.5, 4.6, 7.2]
}

{
"id": 1,
"name": "John",
"notas": [
{
"materia": "extracao dados I",
"valor": 10.0
}
]
}


### Autenticação

Por que uma API utiliza autenticação ?

- Garantir que o usuário só acesse seus próprios dados
- Garantir que o usuário só acesses recursos específicos
- Auditoria de acesso

Principais Tipos de Autenticação

- Baerer Token
- OAuth
- API Key

- Baerer Token
- API Key

Passados no header da requisição


In [ ]:
import json

In [ ]:
def autenticar(login, senha):
    login_response = requests.post(
        url="https://dummyjson.com/auth/login",
        headers={"Content-Type": "application/json"},
        data=json.dumps({
            "username": login,
            "password": senha,
            "expiresInMins": 1
        })
    )
    
    return login_response.json()['token']

access_token = autenticar(login="kminchelle", senha="0lelplR")

In [ ]:
url_base = "https://dummyjson.com/auth/"

authorization_header = {
    "Authorization": f"Bearer {access_token}", 
    "Content-Type": "application/json"
}

In [ ]:
## Caso de Sucesso com Authenticação

authenticated_products_response = requests.get(
    url=f"{url_base}/products",
    headers=authorization_header
)

print(f"Status Code: {authenticated_products_response.status_code}")
print(f"Content: {authenticated_products_response.text}")

In [ ]:
authorization_header_errado = {
    "Authorization": f"Bearer 123654646156475656498789656", 
    "Content-Type": "application/json"
}

authenticated_products_response = requests.get(
    url=f"{url_base}/products",
    headers=authorization_header_errado
)

print(f"Status Code: {authenticated_products_response.status_code}")

## Implementação do Fluxo de Re-autenticação


In [ ]:
def constroi_authentication_headers():
    access_token = autenticar(login="kminchelle", senha="0lelplR")
    
    return {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

In [ ]:
def gerencia_requisicao_authenticada(url: str, authorization_header: dict, max_retries: int, current_retry: int =0):

    response = requests.get(
        url=url,
        headers=authorization_header
    )
    
    try:
        response_content = response.json()
        
        if response.status_code == 401:
            if current_retry < max_retries:
                if response_content['name'] == "TokenExpiredError":
                    return gerencia_requisicao_authenticada(
                        url=url,
                        authorization_header=constroi_authentication_headers(),
                        max_retries=max_retries,
                        current_retry=current_retry+1
                    )
            else:
                print(f"Não foi possível recuperar a requisição após o código. Código: {response.status_code}")
          
        return response_content
    except Exception as ex:
        print(f"Fluxo de exceção. Detalhe: {ex}")

In [ ]:
gerencia_requisicao_authenticada(url=f"{url_base}/products", authorization_header=authorization_header, max_retries=3)

## Acessando apenas os recursos desejados em uma API


Filtrando recursos por Request Path


In [ ]:
def constroi_authentication_headers():
    access_token = autenticar(login="kminchelle", senha="0lelplR")
    
    return {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

In [ ]:
# Query params são definidos pelo caracter "?" e são combinados com outros critérios pelo caractere "&"
# transaction_date
#?transaction_date="2023-09-18"

query_reponse = requests.get(url="https://dummyjson.com/auth/products/search?q=phone", headers=constroi_authentication_headers())
query_reponse.json()

In [ ]:
## Trabalhando com Paginação
## Parametros de Paginação me ajudam e definir qual intervalo de dados eu desejo consumir
## Onde é util:
## 5.000.000
## Lambda 1KB
## 5.000.000 - API pode demorar muito pra responder, a API pode estar programada pra não responder tudo isso, ou programa pode não ter
## Capacidade de processar tudo isso

## 0  Skip
## 10 limit
## 100 total

## 10 skip 
## 10 limit
## 100 total

## 20 skip 
## 10 limit
## 100 total

## 90 skip 
## 10 limit
## 100 total



response_paginado = requests.get("https://dummyjson.com/auth/products?limit=10&skip=10&select=title,price", headers=constroi_authentication_headers())

response_paginado.json()